In [1]:
%load_ext autoreload

In [2]:
%autoreload
import numpy as np
import scipy.special as sp
from scipy.integrate import quad
import darkhistory.physics as phys

In [3]:
sp.bernoulli(5)

array([ 1.        , -0.5       ,  0.16666667,  0.        , -0.03333333,  0.        ])

In [4]:
def F1(a, b):
    def indef_int(x):
        return x*np.log(1 - np.exp(-x)) - sp.spence(1 - np.exp(-x))
    return indef_int(b) - indef_int(a)

def F0(a,b):
    def indef_int(x):
        return np.log(1 - np.exp(-x))
    return indef_int(b) - indef_int(a)



In [5]:
def H(a, b, tol=1):
    
    bound = 2
    
    def low_summand(x):
        k = 1
        while True:
            if k == 1:
                next_term = -1/x - (1/2)*np.log(x)
                k += 1
            else:
                next_term = (sp.bernoulli(k)[-1]*(x**(k-1))/
                        (sp.factorial(k)*(k-1))
                )
                k += 2
            yield next_term
                
    def high_summand(x):
        k = 1
        while True:
            next_term = sp.expn(1,k*x)
            k += 1
            yield next_term
        
    err = 10*tol
    
    if a >= bound and b >= bound:
        high_sum_a = high_summand(a)
        high_sum_b = high_summand(b)
        integral = next(high_sum_a) - next(high_sum_b)
    
        while err > tol:
            next_term = next(high_sum_a) - next(high_sum_b)
            err = np.abs(next_term/integral)
            integral += next_term
    
    elif a < bound and b >= bound:
        low_sum_a = low_summand(a)
        high_sum_b = high_summand(b)
        low_sum_bound = low_summand(bound)
        int_bound_inf = 0.053082306482669888568
        int_a_bound = next(low_sum_bound) - next(low_sum_a)
        int_bound_b = int_bound_inf - next(high_sum_b)
        integral = int_a_bound + int_bound_b
        
        while err > tol:
            next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
            next_term_bound_b = - next(high_sum_b)
            next_term = next_term_a_bound + next_term_bound_b
            err = np.abs(next_term/integral)
            integral += next_term
    
    else:
        low_sum_a = low_summand(a)
        low_sum_b = low_summand(b)
        integral = next(low_sum_b) - next(low_sum_a)
        
        while err > tol:
            next_term = next(low_sum_b) - next(low_sum_a)
            err = np.abs(next_term/integral)
            integral += next_term
    
    return integral
            

In [6]:
print(H(0.02124,1.9033,tol=1e-12))
print(H(1e-6,1e6,tol=1e-12))
print(H(4.50204,9.10294,tol=1e-12))

44.4616483972
999992.461914
0.00206974344822


In [7]:
def G(a, b, tol=1):
    
    bound = 2
    
    def low_summand(x):
        k = 1
        while True:
            if k == 1:
                next_term = (1/2)*(np.log(x)**2) - (x/2)*(np.log(x) - 1)
                k += 1
            else:
                next_term = (sp.bernoulli(k)[-1]*(x**k)/
                        (sp.factorial(k)*(k**2))*(k*np.log(x) - 1)
                )
                k += 2
            yield next_term
                
    def high_summand(x):
        k = 1
        while True:
            next_term = (1/k)*(np.exp(-k*x)*np.log(x) + sp.expn(1,k*x))
            k += 1
            yield next_term
        
    err = 10*tol
    
    if a >= bound and b >= bound:
        high_sum_a = high_summand(a)
        high_sum_b = high_summand(b)
        integral = next(high_sum_a) - next(high_sum_b)
    
        while err > tol:
            next_term = next(high_sum_a) - next(high_sum_b)
            err = np.abs(next_term/integral)
            integral += next_term
    
    elif a < bound and b >= bound:
        low_sum_a = low_summand(a)
        high_sum_b = high_summand(b)
        low_sum_bound = low_summand(bound)
        int_bound_inf = 0.15171347859984083704
        int_a_bound = next(low_sum_bound) - next(low_sum_a)
        int_bound_b = int_bound_inf - next(high_sum_b)
        integral = int_a_bound + int_bound_b
        
        while err > tol:
            next_term_a_bound = next(low_sum_bound) - next(low_sum_a)
            next_term_bound_b = - next(high_sum_b)
            next_term = next_term_a_bound + next_term_bound_b
            err = np.abs(next_term/integral)
            integral += next_term
    
    else:
        low_sum_a = low_summand(a)
        low_sum_b = low_summand(b)
        integral = next(low_sum_b) - next(low_sum_a)
        
        while err > tol:
            next_term = next(low_sum_b) - next(low_sum_a)
            err = np.abs(next_term/integral)
            integral += next_term
    
    return integral
            

In [8]:
print(G(0.02124,1.9033,tol=1e-12))
print(G(1e-6,1e6,tol=1e-12))
print(G(4.50204,9.10294,tol=1e-12))

-6.90361462464
-94.7054794794
0.0185904820982


# Check ICS Calculations

In [9]:
eleceng = 3.45e4+phys.me
photeng = 3.789
T = phys.TCMB(1)*1000

gamma = eleceng/phys.me
beta = np.sqrt(1 - 1/gamma**2)

prefac = (phys.c*(3/8)*phys.thomson_xsec/(2*gamma**3*beta**2)
          *(8*np.pi/(phys.ele_compton*phys.me)**3)
         )

minratio = (1-beta)/(1+beta)*photeng/T
maxratio = (1+beta)/(1-beta)*photeng/T

fac1 = (1+beta**2)/beta**2*np.sqrt((1+beta)/(1-beta))*F1(minratio,photeng/T)*T**2
fac2 = (2/beta)*np.sqrt((1-beta)/(1+beta))*photeng*F0(minratio,photeng/T)*T
fac3 = -(1-beta)**2/(beta**2)*np.sqrt((1+beta)/(1-beta))*photeng**2*H(minratio,photeng/T)
fac4 = 2/(gamma*beta**2)*photeng*np.log((1-beta)/(1+beta)*photeng)*F0(minratio,photeng/T)*T
fac5 = -2/(gamma*beta**2)*photeng*(G(minratio,photeng/T)*T + T*np.log(T)*F0(minratio,photeng/T))

fac6 = -(1+beta**2)/beta**2*np.sqrt((1-beta)/(1+beta))*F1(photeng/T,maxratio)*T**2
fac7 = (2/beta)*np.sqrt((1+beta)/(1-beta))*photeng*F0(photeng/T,maxratio)*T
fac8 = (1+beta)/(gamma*beta**2)*photeng**2*H(photeng/T,maxratio)
fac9 = -2/(gamma*beta**2)*photeng*np.log((1+beta)/(1-beta)*photeng)*F0(photeng/T,maxratio)*T
fac10 = 2/(gamma*beta**2)*photeng*(G(photeng/T,maxratio)*T + T*np.log(T)*F0(photeng/T,maxratio))

part_low = fac1+fac2+fac3+fac4+fac5
part_high = fac6+fac7+fac8+fac9+fac10
# print(prefac*part_low)
# print(prefac*part_high)
print(prefac*part_low + prefac*part_high)
print(prefac*fac1 + prefac*fac6)
print(prefac*fac2 + prefac*fac7)
print(prefac*fac3 + prefac*fac8)
print(prefac*(fac4+fac5+fac9+fac10))

1.64486059306e-05
0.000898496647561
0.00049682352617
-0.00115790369271
-0.00022096787509


In [10]:
print(minratio)

7.76337859272


In [11]:
phys.ele_compton

2.4263102352464287e-10